In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import math
import warnings 
warnings.filterwarnings('ignore')

import statsmodels as sm 
import statsmodels.api as smt
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv('UnderemploymentRate_InSample.csv')
df.head()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

In [ ]:
# Add missing value：1978-01-01 7.5
new_row = pd.DataFrame({'Date':['1978-01-01'],'Underemployment_Rate': [7.5]})
df = pd.concat([new_row, df])
df['Date'] = pd.to_datetime(df['Date'], dayfirst = True)

df.set_index('Date', inplace=True)
df.head()

# 2.0 EDA

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
# Time-series plot
ts = df['Underemployment_Rate']

plt.figure(figsize=(15, 6))
plt.plot(ts)
plt.title('Time Series Plot', fontsize=20)
plt.xlabel('Date')
plt.ylabel('Underemployment Rate')
plt.show()

In [ ]:
# CMA
CMA_5 = ts.rolling(5,center=True).mean()
CMA_12 = ts.rolling(12,center=True).mean().rolling(2,center=True).mean().shift(-1)

de_trend = ts - CMA_12

plt.figure(figsize=(15, 6))
plt.plot(ts.index, ts, label='Original Plot')
plt.plot(ts.index, de_trend, label='Reveal Seasonality')
plt.plot(ts.index, CMA_5, label='CMA-5')
plt.plot(ts.index, CMA_12, label='CMA-12')

plt.xlabel('Date')
plt.ylabel('UnderemploymentRate')
plt.legend()
plt.show()

In [ ]:
# ACP & PACF
smt.graphics.tsa.plot_acf(ts, lags=30, alpha = 0.05)
smt.graphics.tsa.plot_pacf(ts, lags=30, alpha = 0.05)
plt.show()

In [ ]:
# Calculate difference series
underemployment_diff = pd.Series.diff(ts)
# Discard the nan value
underemployment_diff = underemployment_diff.dropna()

# Plot the differenced data
plt.figure(figsize=(15,6))
plt.plot(underemployment_diff)
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Monthly Underemployment Rate Differencing')
plt.show()

In [ ]:
smt.graphics.tsa.plot_acf(underemployment_diff, lags=30, alpha = 0.05)
smt.graphics.tsa.plot_pacf(underemployment_diff, lags=30, alpha = 0.05)
plt.show()

In [ ]:
ts_train = ts[:-24]
ts_valid = ts[-24:]

# 3.0 Model Training 

## 3.1 Decomposition

In [ ]:
print(ts_train.shape)

In [ ]:
456/12

In [ ]:
CMA12 = ts_train.rolling(12, center=True).mean().rolling(2, center=True).mean().shift(-1)
ts_res = ts_train - CMA12
ts_res_zero = np.nan_to_num(ts_res)

monthly_S = np.reshape(ts_res_zero, (-1, 12))  

monthly_avg = np.mean(monthly_S, axis=0)

seasonal_idx = monthly_avg.mean()
seasonal_idx_normalized = monthly_avg - seasonal_idx

seasonal = np.tile(seasonal_idx_normalized, len(ts_train) // 12 + 1)[:len(ts_train)]  

# Ensure that the length of the seasonality index is consistent with the length of the time series
seasonally_adjusted = ts_train - seasonal

### 3.1.1 Decomposition_Linear_Model

In [ ]:
X = np.linspace(1, len(seasonally_adjusted), len(seasonally_adjusted))  
X = X.reshape(-1, 1)  

y = seasonally_adjusted.values
tm = LinearRegression()
tm.fit(X, y)

X_fitting = np.reshape(np.arange(1, len(seasonally_adjusted) + 1), (len(seasonally_adjusted), 1))
trend_linear = tm.predict(X_fitting)

plt.figure(figsize=(15, 6))

plt.plot(ts_train.index, seasonally_adjusted)
plt.plot(ts_train.index, trend_linear)

plt.title("Final Trend Estimation Linear Regression")
plt.xlabel('Date')
plt.ylabel('Underemployement Rate')

plt.legend(["Seasonally Adjusted Data", "Trend Component (Linear Regression)"])
plt.show()

In [ ]:
X_pred = np.arange(len(seasonally_adjusted) + 1, len(seasonally_adjusted) + 25).reshape(-1, 1)
Trend_pred_linear = tm.predict(X_pred)

deli = Trend_pred_linear + seasonal[:24]

plt.figure(figsize=(15,6))
plt.plot(ts.index, ts, label='Original')
plt.plot(ts_valid.index, deli, label='Trend + Seasonal')

plt.title('Final Trend Estimation Linear Regression')
plt.xlabel('Date')
plt.ylabel('Underemployment Rate')

plt.legend()
plt.show()

### 3.1.2 Decomposition_Linear_Model RMSE

In [ ]:
linear_mse = mean_squared_error(ts_valid,deli)
Dep_linear_rmse = np.sqrt(linear_mse)
print('RMSE: {0:.4f}'.format(Dep_linear_rmse))

### 3.1.3 Decomposition_Polynomial_Model

In [ ]:
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

tm2 = LinearRegression()
tm2.fit(X_poly, y)

X_fitting_p = np.reshape(np.arange(1, len(seasonally_adjusted) + 1), (-1, 1))
X_fitting_p_poly = poly.transform(X_fitting_p)

trend_linear = tm2.predict(X_fitting_p_poly)

plt.figure(figsize=(15, 6))
plt.plot(ts_train.index, seasonally_adjusted)
plt.plot(ts_train.index, trend_linear)

plt.title('Final Trend Estimation Linear Regression')
plt.xlabel('Date')
plt.ylabel('Underemployment Rate')

plt.legend(['Seasonally Adjusted Data', 'Trend Component (Linear Regression)'])
plt.show()

In [ ]:
X_pred = np.arange(len(seasonally_adjusted) + 1, len(seasonally_adjusted) + 25).reshape(-1, 1)
X_pred_poly = poly.transform(X_pred)

Trend_pred = tm2.predict(X_pred_poly)
depo = Trend_pred + seasonal[:24]

plt.figure(figsize=(15,6))
plt.plot(ts.index, ts, label='Original')
plt.plot(ts_valid.index, depo, label='Trend + Seasonal')

plt.title('Final Trend Estimation Linear Regression')
plt.xlabel('Date')
plt.ylabel('Underemployment Rate')

plt.legend()
plt.show()

## 3.1.3 Decomposition_Polynomial_Model_RMSE

In [ ]:
Dep_mse = mean_squared_error(ts_valid, depo)
Dep_poly_rmse = np.sqrt(Dep_mse)

print('RMSE: {0:.4f}'.format(Dep_poly_rmse))

## 3.2 Linear Model prediction

In [ ]:
X_train = ts_train.index.to_julian_date().values.reshape(-1, 1)
y_train = ts_train.values

model = LinearRegression()
model.fit(X_train, y_train)

X_valid = ts_valid.index.to_julian_date().values.reshape(-1, 1)

y_pred = model.predict(X_valid)

In [ ]:
plt.figure(figsize=(15, 6)) 

plt.plot(ts.index, ts.values, label='Original')

plt.plot(ts_valid.index, y_pred, label='Predicted Data')

plt.xlabel('Date')
plt.ylabel('Underemployment Rate')
plt.title('Original Data vs Predicted Data')
plt.legend()
plt.show()

### 3.2.1 Linear Model RMSE

In [ ]:
mse = mean_squared_error(ts_valid.values, y_pred)
linear_rmse = np.sqrt(mse)
print('LR Predicted Model RMSE: {0:.4f}'.format(linear_rmse))

## 3.3 Holt-Winters Model

### 3.3.1 Simple exponential smoothing

In [ ]:
lt_pandas1 = ts.ewm(alpha=0.05, adjust=False).mean()
lt_pandas2 = ts.ewm(alpha=0.1, adjust=False).mean()
lt_pandas3 = ts.ewm(alpha=0.3, adjust=False).mean()
lt_pandas4 = ts.ewm(alpha=0.7, adjust=False).mean()

plt.figure(figsize=(16,8))
plt.plot(ts, label='Original data')
plt.plot(lt_pandas1, label = "Alpha = 0.05")
plt.plot(lt_pandas2, label = "Alpha = 0.1")
plt.plot(lt_pandas3, label = "Alpha = 0.3")
plt.plot(lt_pandas4, label = "Alpha = 0.7")
plt.xlabel('Month')
plt.ylabel('Number of Unemployment')
plt.title("SES smoothing with various values of Alpha")
plt.legend()
plt.show()
# plt.savefig('SES_holt')

### 3.3.2 Multiplicative model

In [ ]:
fit_mul = ExponentialSmoothing (ts_train, seasonal_periods=12, trend='add', seasonal= 'mul').fit()

smooth_mul = fit_mul.fittedvalues

plt.figure(figsize=(15,6))
plt.plot(ts_train[1:], label = 'Original data')
plt.plot(smooth_mul, 'g--', label = 'Multiplicative Holt-Winters')
plt.xlabel('Date')
plt.ylabel('Unemployment Rate')
plt.title('Multiplicative Holt-Winters Method' )
plt.legend()
plt.show()
# plt.savefig('holt_model')

In [ ]:
# symbol r $ and \ in the results variable are the latex symbols for visualization in notebook
multi_co = pd.DataFrame(index=[r"$\alpha$",\
                              r"$\beta$",\
                              r"$\gamma$",\
                              r"$l_0$",\
                              "$b_0$",\
                              "SSE"])
# ExponentialSmoothing() object has following attributes
params_mul_hot = ['smoothing_level', \
          'smoothing_trend', \
          'smoothing_seasonal', \
          'initial_level', \
          'initial_trend']

# check out the performance of additive and multiplicative
multi_co["Multiplicative"] = [fit_mul.params[p] for p in params_mul_hot] + [fit_mul.sse]
multi_co

### 3.3.3 Holt_Winters' Forecasting

In [ ]:
# Forecast 24 more data points
y_mul = fit_mul.forecast(24)

# And plot al together
plt.figure(figsize=(16,5))
plt.plot(ts_train, label = 'In-sample')
plt.plot(ts_valid, label = 'Validation')
plt.plot(y_mul, '--g',label = 'Multiplicative Holt-Winters')
plt.axvline(x=ts.index[len(ts_train)],color='black',alpha=0.5)

plt.xlabel("Date")
plt.ylabel("Underemployment Rate")
plt.title("Forecast Holt-Winters Methods")
plt.legend()
plt.show()
# plt.savefig('holt_fore')

### 3.3.4  Holt_winter RMSE

In [ ]:
hot_mse = mean_squared_error(ts_valid, y_mul)
hot_mse

hot_rmse = math.sqrt(hot_mse)
print('HW RMSE: {0:.4f}'.format(hot_rmse))

## 3.4 ARIMA

### 3.4.1 Stationary

In [ ]:
plt.figure(figsize=(16,5))
 
# Original Series
fig, (ax1, ax2, ax3) = plt.subplots(3)
ax1.plot(ts); ax1.set_title('Original Series'); ax1.axes.xaxis.set_visible(False)
# 1st Differencing
ax2.plot(underemployment_diff); ax2.set_title('1st Order Differencing'); ax2.axes.xaxis.set_visible(False)
# 2nd Differencing
ax3.plot(underemployment_diff.diff()); ax3.set_title('2nd Order Differencing')
plt.show()
# plt.savefig('arima_diff')

In [ ]:
# from statsmodels.tsa.stattools import adfuller
adf = adfuller(ts)
print('ADF Statistic: %f' % adf[0])
print('p-value: %f' % adf[1])
print('Critical Values:')
for key, value in adf[4].items():
  print('\t%s: %.3f' % (key, value))

#the p-value is more than 0.05 this means our null hypothesis will be 
# rejected and we will take this series as non-stationary.

In [ ]:
from statsmodels.tsa.stattools import adfuller
adf = adfuller(underemployment_diff)
print('ADF Statistic: %f' % adf[0])
print('p-value: %f' % adf[1])
print('Critical Values:')
for key, value in adf[4].items():
  print('\t%s: %.3f' % (key, value))

#the p-value is less than 0.05 this means our null hypothesis will be 
# hold and we will take this series as stationary.

### 3.4.2 ACF

In [ ]:
plt.figure(figsize=(16,5))

# from statsmodels.graphics.tsaplots import plot_acf
fig, (ax1, ax2, ax3) = plt.subplots(3)
plot_acf(ts, lags=50, alpha = 0.05, ax=ax1)
ax1.axes.xaxis.set_visible(False)
ax1.set_title('Autocorrelation')

plot_acf(underemployment_diff.dropna(),lags=50, alpha = 0.05, ax=ax2)
ax2.set_title('Autocorrelation - 1st order diff')
ax2.axes.xaxis.set_visible(False)

plot_acf(underemployment_diff.diff().diff().dropna(), 
         lags=50, alpha = 0.05, ax=ax3)
ax3.set_title('Autocorrelation - 2st order diff')
plt.show()
# plt.savefig('arima_acf_diff')

### 3.4.3 PACF

In [ ]:
smt.graphics.tsa.plot_pacf(underemployment_diff.dropna(),lags=50,alpha=0.05)

plt.show()
# plt.savefig('arima_pcf')

PACF cut-off (p): 2 or 1
Here we can see that the second lag is significantly out of the limit and the third one is also out of the significant limit but it is not that far so we can select the order of the p as 2.

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

p=2
d=1
q=0
model_ari_210 = sm.tsa.arima.model.ARIMA(ts_train, order = (p,d,q))
ari210 = model_ari_210.fit()

ari210.summary()

In [ ]:
ari210_for = ari210.forecast(24)

# And plot al together
plt.figure(figsize=(16,5))
plt.plot(ts_train, label = 'In-sample')
plt.plot(ts_valid, label = 'Validation')
plt.plot(ari210_for, '--g',label = 'ARIMA (2,1,0) fitted values')
plt.axvline(x=ts.index[len(ts_train)],color='black',alpha=0.5)

plt.xlabel("Date")
plt.ylabel("Underemployment Rate")
plt.title("ARIMA(2,1,0) Model Fitted vs True Values")
plt.legend()
plt.show()
# plt.savefig('arima_210')

### 3.4.4 ARIMA (1,1,0)


In [ ]:
from statsmodels.tsa.arima_model import ARIMA

p=1
d=1
q=0
model_ari_110 = sm.tsa.arima.model.ARIMA(ts_train, order = (p,d,q))
ari110 = model_ari_110.fit()

ari110.summary()

In [ ]:
ari110_for = ari110.forecast(24)

# Plot ala together
plt.figure(figsize=(16,5))
plt.plot(ts_train, label = 'In-sample')
plt.plot(ts_valid, label = 'Validation')
plt.plot(ari110_for, '--g',label = 'ARIMA (1,1,0) fitted values')
plt.axvline(x=ts.index[len(ts_train)],color='black',alpha=0.5)

plt.xlabel("Date")
plt.ylabel("Underemployment Rate")
plt.title("ARIMA(1,1,0) Model Fitted vs True Values")
plt.legend()
plt.show()
# plt.savefig('arima_110')

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

p=2
d=1
q=1
model_ari_211 = sm.tsa.arima.model.ARIMA(ts_train, order = (p,d,q))
ari211 = model_ari_211.fit()

ari211_for = ari211.forecast(24)

# And plot all together
plt.figure(figsize=(16,5))
plt.plot(ts_train, label = 'In-sample')
plt.plot(ts_valid, label = 'Validation')
plt.plot(ari211_for, '--g',label = 'ARIMA (2,1,1) fitted values')
plt.axvline(x=ts.index[len(ts_train)],color='black',alpha=0.5)

plt.xlabel("Date")
plt.ylabel("Underemployment rate")
plt.title("ARIMA(2,1,1) model fitted vs true values")
plt.legend()
plt.show()
# plt.savefig('arima_211')

### 3.4.5 Using AIC to select best fitting (p,q) order of ARIMA(p,d,q) models

In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Load your time series data
# Assuming your data is in 'series'

# Define the range of p, d, q values to consider
p_values = range(5)
d_values = range(5)
q_values = range(5)

# Initialize variables to store best values and AIC
best_aic = float("inf")
best_pdq = None

# Loop through all combinations of p, d, q
for p in p_values:
    for d in d_values:
        for q in q_values:
            try:
                model_aic = ARIMA(ts_train, order=(p, d, q))
                results_aic = model_aic.fit()
                aic = results_aic.aic
                if aic < best_aic:
                    best_aic = aic
                    best_pdq = (p, d, q)
            except:
                continue

print("Best AIC:", best_aic)
print("Best (p, d, q):", best_pdq)

/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning

/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarni

/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarni

/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarni

Best AIC: 285.55758124424904
Best (p, d, q): (4, 1, 2)


/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [ ]:
from statsmodels.tsa.arima_model import ARIMA

p=4
d=1
q=2
model_ari2 = sm.tsa.arima.model.ARIMA(ts_train, order = (p,d,q))
ari2 = model_ari2.fit()
ari2.summary()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

p=4
d=1
q=2
model_ari_412 = sm.tsa.arima.model.ARIMA(ts_train, order = (p,d,q))
ari412 = model_ari_412.fit()

ari412_for = ari412.forecast(24)

# And plot al together
plt.figure(figsize=(16,5))
plt.plot(ts_train, label = 'In-sample')
plt.plot(ts_valid, label = 'Validation')
plt.plot(ari412_for, '--g',label = 'ARIMA (4,1,2) fitted values')
plt.axvline(x=ts.index[len(ts_train)],color='black',alpha=0.5)

plt.xlabel("Date")
plt.ylabel("Underemployment rate")
plt.title("ARIMA(4,1,2) model fitted vs true values")
plt.legend()
plt.show()
# plt.savefig('arima_412_aic')

### 3.4.6 ARIMA RMSE

In [ ]:
import math
ari_rmse = math.sqrt(mean_squared_error(ts_valid, ari412_for))
print('RMSE: {0:.4f}'.format(ari_rmse))

## 3.5 SARIMA

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
# SARIMAX(2,1,2)
model_sarima1 = SARIMAX(ts_train, 
                order=(2,1,2), 
                seasonal_order=(1,1,1, 12),
                enforce_stationarity=False,
                enforce_invertibility=False)

# Fit the SARIMAX model 
model_sarima1_fit = model_sarima1.fit(disp = -1)

# Forecast 
model_sarima1_forecast = model_sarima1_fit.forecast(24)

In [ ]:
model_sarima1_forecast

In [ ]:
print(model_sarima1_fit.summary())

In [ ]:
# Plot forecast and true values
plt.figure(figsize=(15,5))
plt.plot(ts_train,label='Train')
plt.plot(ts_valid,'b',label='Valid_true')

plt.plot(ts_valid.index, model_sarima1_forecast,'r',label='Valid_forecast')
plt.title("SARIMA(2,1,2)")
plt.legend()
plt.axvline(x=ts.index[len(ts_train)],color='black')  # Make a vertical line indicating train/test separation
plt.show()

In [ ]:
sarima1_rmse = mean_squared_error(ts_valid,model_sarima1_forecast)**0.5
print('RMSE: {0:.4f}'.format(sarima1_rmse))

In [ ]:
#SARIMAX(1,1,2)
model_sarima2 = SARIMAX(ts_train, 
                order=(1,1,2), 
                seasonal_order=(1,1,1, 12),
                enforce_stationarity=False,
                enforce_invertibility=False)

# Fit the SARIMAX model 
model_sarima2_fit = model_sarima2.fit(disp = -1)

# Forecast 
model_sarima2_forecast = model_sarima2_fit.forecast(24)

In [ ]:
model_sarima2_forecast

In [ ]:
print(model_sarima2_fit.summary())

In [ ]:
# Plot forecast and true values
plt.figure(figsize=(15,5))
plt.plot(ts_train,label='Train')
plt.plot(ts_valid,'b',label='Valid_true')

plt.plot(ts_valid.index, model_sarima2_forecast,'r',label='Valid_forecast')
plt.title("SARIMA(1,1,2)")
plt.xlabel('Date')
plt.ylabel('Underemployment Rate')
plt.legend()
plt.axvline(x=ts.index[len(ts_train)],color='black')  # Make a vertical line indicating train/test separation
plt.show()

In [ ]:
sarima2_rmse = mean_squared_error(ts_valid,model_sarima2_forecast)**0.5
print('RMSE: {0:.4f}'.format(sarima2_rmse))

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
# Define the range of p, d, q values to consider
p_range = range(0, 3)  # Autoregressive order
d_range = range(1, 2)  # Differencing order
q_range = range(0, 3)  # Moving average order

# Seasonal parameters
P_range = range(0, 2)  # Seasonal autoregressive order
D_range = range(0, 2)  # Seasonal differencing order
Q_range = range(0, 2)  # Seasonal moving average order
S = 12                 # Seasonal periodicity (e.g., 12 for monthly data)

# Initialize variables to store best values and AIC
best_aic = float("inf")
best_params = None

# Loop through all combinations of p, d, q
for p in p_range:
    for d in d_range:
        for q in q_range:
            for P in P_range:
                for D in D_range:
                    for Q in Q_range:
                        try:
                            # Define the SARIMA model
                            model = SARIMAX(ts_train, order=(p, d, q), 
                                            seasonal_order=(P, D, Q, S),
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
                            # Fit the model
                            results = model.fit(disp=0)  # Set disp=0 to turn off debugging information
                            
                            # Check if the current model has a lower AIC than the previous best
                            if results.aic < best_aic:
                                best_aic = results.aic
                                best_params = (p, d, q, P, D, Q)
                        except Exception as e:
                            print(f"Failed to fit model with parameters (p={p}, d={d}, q={q}, P={P}, D={D}, Q={Q}): {str(e)}")
                            continue


/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency in

/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/soooniaxx/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency in

In [ ]:
print("Best AIC: {:.4f}".format(best_aic))
print("Best Parameters: p={}, d={}, q={}, P={}, D={}, Q={}".format(*best_params))

In [ ]:
#SARIMAX(Best AIC)
model_sarima3 = SARIMAX(ts_train, 
                order=(2,1,2), 
                seasonal_order=(1,0,1, 12),
                enforce_stationarity=False,
                enforce_invertibility=False)

# Fit the SARIMAX model 
model_sarima3_fit = model_sarima3.fit(disp = -1)

# Forecast 
model_sarima3_forecast = model_sarima3_fit.forecast(24)

# Plot forecast and true values
plt.figure(figsize=(15,5))
plt.plot(ts_train,label='Train')
plt.plot(ts_valid,'b',label='Valid_true')

plt.plot(ts_valid.index, model_sarima3_forecast,'r',label='Valid_forecast')
plt.title("SARIMA(Best AIC)")
plt.xlabel('Date')
plt.ylabel('Underemployment Rate')
plt.legend()
plt.axvline(x=ts.index[len(ts_train)],color='black')  # Make a vertical line indicating train/test separation
plt.show()

In [ ]:
sarima3_rmse = mean_squared_error(ts_valid,model_sarima3_forecast)**0.5
print('RMSE: {0:.4f}'.format(sarima3_rmse))

### 3.5.1 SARIMA RMSE

In [ ]:
print(sarima1_rmse,sarima2_rmse,sarima3_rmse)

## 3.6 Feed Forward NN Model (FNN)

In [ ]:
import tensorflow as tf
import random 

np.random.seed(0)
tf.random.set_seed(0)
random.seed(0)

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [ ]:
# Drop all Nans
data_not_scaled = df.dropna()               
# Convert from DataFrame to Python Array
data_not_scaled = data_not_scaled.values         
# Make sure the data is type of float
data_not_scaled = data_not_scaled.astype('float')  

# Plot the time series
plt.figure(figsize=(15,6))
plt.plot(df.index,data_not_scaled)
plt.xlabel('Date')
plt.ylabel('Underemployment Rate')
plt.show()

In [ ]:
# Time window to define the number of features in each row
time_window = 12

# Define a scaler object
scaler = MinMaxScaler(feature_range=(0, 1))

# Compute in-sample size
train_size = len(data_not_scaled) - 24
valid_size = 24

# Fitting the scaler
fitted_transformer = scaler.fit(data_not_scaled[:train_size+time_window])

# Transforming using trained scaler
data_scaled = fitted_transformer.transform(data_not_scaled)

# Plot the scaled data
plt.figure(figsize=(15,6))
plt.plot(df.index,data_scaled)
plt.xlabel('Date')
plt.ylabel('Underemployment Rate in 0-1 scale')
plt.title('Transformed data')
plt.show()

In [ ]:
# Pre-allocation
Xall, Yall = [], []
Xall_not_scaled, Yall_not_scaled = [], []

# Use each rolling window as a row in the data matrix 
for i in range(time_window, len(data_scaled)):
    Xall.append(data_scaled[i-time_window:i, 0]) # Access the accuracy
    Xall_not_scaled.append(data_not_scaled[i-time_window:i, 0]) # Training NN
    Yall.append(data_scaled[i, 0])
    Yall_not_scaled.append(data_not_scaled[i, 0])

# Convert them from list to numpy array
Xall = np.array(Xall)
Yall = np.array(Yall)
Xall_not_scaled = np.array(Xall_not_scaled)
Yall_not_scaled = np.array(Yall_not_scaled)

# Training data & Validation data
Xtrain = Xall[:train_size, :]
Ytrain = Yall[:train_size]

Xvalid = Xall[-valid_size:,:]
Yvalid = Yall[-valid_size:]

In [ ]:
Xtrain.shape, Ytrain.shape

In [ ]:
Xvalid.shape, Yvalid.shape

### 3.6.1 FNN Model Building Process

In [ ]:
# Define FNN model architecture
model_FNN = Sequential()
model_FNN.add(Dense(24, input_dim = time_window, activation='relu'))
model_FNN.add(Dense(16, input_dim = time_window, activation='relu'))
model_FNN.add(Dense(1))

In [ ]:
# Compile the model
model_FNN.compile(loss='mean_squared_error', optimizer='adam')
model_FNN.summary()

In [ ]:
# Train the model
model_FNN.fit(Xtrain, Ytrain, epochs=300, batch_size=20 , verbose=1)

In [ ]:
# 24-step Prediction
dynamic_prediction = np.copy(data_scaled[:len(data_scaled) - valid_size])

for i in range(len(data_scaled) - valid_size, len(data_scaled)):
    last_feature = np.reshape(dynamic_prediction[i-time_window:i], (1,time_window,1))
    next_pred = model_FNN.predict(last_feature)
    dynamic_prediction = np.append(dynamic_prediction, next_pred)

# Transform forecast values to original scale
dynamic_prediction = dynamic_prediction.reshape(-1,1)
dynamic_prediction_original_scale = scaler.inverse_transform(dynamic_prediction)

# Plot
test_index = np.arange(len(data_scaled) - valid_size, len(data_scaled), 1)

plt.figure(figsize=(15,6))
plt.plot(scaler.inverse_transform(data_scaled[:len(data_scaled) - valid_size]), label='Training Data')
plt.plot(test_index, scaler.inverse_transform(data_scaled[-valid_size:]), label='True Validation Data') 
plt.plot(test_index, dynamic_prediction_original_scale[-valid_size:], label='Validation Prediction') 
plt.xlabel('Date')
plt.ylabel('Underemployment Rate')
plt.legend(loc = "upper left")
plt.show()

### 3.6.2 FNN RMSE

In [ ]:
# Compute RMSE score on Predicted validation data
FNN_RMSE = np.sqrt(mean_squared_error(Yvalid, dynamic_prediction[-valid_size:]))
print('24-step Forecast RMSE for FNN Model: {0:.4f}'.format(FNN_RMSE))
FNN_RMSE = FNN_RMSE

## 3.7 RMSE Presenting

In [ ]:
rmse_data = {
    'Model': ['SARIMA','ARIMA', 'Holt-Winters', 'Decomposition Linear', 'Linear', 'FNN'],
    'RMSE': [sarima2_rmse, ari_rmse, hot_rmse, Dep_linear_rmse, linear_rmse,FNN_RMSE]
}

gg = pd.DataFrame(rmse_data)

gg_sorted = gg.sort_values(by='RMSE')

print(gg_sorted)

plt.figure(figsize=(14, 6))

bars_top3 = plt.barh(gg_sorted['Model'][:3], gg_sorted['RMSE'][:3], color='orange', label='Top 3')
bars_rest = plt.barh(gg_sorted['Model'][3:], gg_sorted['RMSE'][3:], color='skyblue')

for bar in bars_top3:
    width = bar.get_width()
    plt.text(width, bar.get_y() + bar.get_height()/2, f'{width:.4f}', ha='left', va='center')
    
for bar in bars_rest:
    width = bar.get_width()
    plt.text(width, bar.get_y() + bar.get_height()/2, f'{width:.4f}', ha='left', va='center')

plt.xlabel('RMSE')
plt.title('RMSE Comparison')
plt.grid(axis='x')
plt.show()
# plt.savefig('RMSE Comparision_full')

# 4.0 Model Evalutation 

In [ ]:
df2 = pd.read_csv('UnderemploymentRate_OutofSample.csv')
df2['Date'] = pd.to_datetime(df2['Date'], dayfirst = True)

df2.set_index('Date', inplace=True)
df2.head()

## 4.1 Holt-Winters Model

In [ ]:
fit_mul2 = ExponentialSmoothing (ts, seasonal_periods=12, trend='add', seasonal= 'mul').fit()

smooth_mul2 = fit_mul2.fittedvalues

y_mul2 = fit_mul2.forecast(24)

# And plot al together
plt.figure(figsize=(16,5))
plt.plot(df2, label = 'Out-sample')
plt.plot(ts, label = 'In-sample')
plt.plot(y_mul2, '--g',label = 'Holt-Winters Multiplicative')
plt.axvline(x=ts.index[-1],color='black',alpha=0.5)

plt.xlabel("Date")
plt.ylabel("Underemployment rate")
plt.title(" Forecasts from Holt-Winters'methods")
plt.legend()
plt.show()
# plt.savefig('Forecasts from Hot')

In [ ]:
# hot_mse2 = mean_squared_error(df2, y_mul2)
hot_rmse2 = np.sqrt(mean_squared_error(df2, y_mul2))
hot_rmse2

## 4.2 Seasonal ARIMA

In [ ]:
#SARIMAX model 2
model_sarima22 = SARIMAX(ts, 
                order=(1,1,2), 
                seasonal_order=(1,1,1, 12),
                enforce_stationarity=False,
                enforce_invertibility=False)

# Fit the SARIMAX model 
model_sarima22_fit = model_sarima22.fit(disp = -1)

# Forecast 
model_sarima22_forecast = model_sarima22_fit.forecast(24)

# Plot forecast and true values
plt.figure(figsize=(16,5))
plt.plot(df2, label = 'Out-sample')
plt.plot(ts, label = 'In-sample')
plt.plot(model_sarima22_forecast, '--g',label = 'Seasonal ARIMA')
plt.axvline(x=ts.index[-1],color='black',alpha=0.5)

plt.xlabel("Date")
plt.ylabel("Underemployment rate")
plt.title(" Forecasts from Seasonal ARIMA")
plt.legend()
plt.show()

In [ ]:
# hot_mse2 = mean_squared_error(df2, y_mul2)
sarima_rmse22 = np.sqrt(mean_squared_error(df2, model_sarima22_forecast))
sarima_rmse22

## 4.3 FNN

In [ ]:
model_FNN.fit(Xall, Yall, epochs=300, batch_size=20 , verbose=1)

In [ ]:
# model_FNN.fit(Xall, Yall, epochs=300, batch_size=20 , verbose=1)

#Dynamical Prediction
dynamic_prediction = np.copy(data_scaled[:len(data_scaled) - valid_size])

for i in range(len(data_scaled) - valid_size, len(data_scaled)):
    last_feature = np.reshape(dynamic_prediction[i-time_window:i], (1,time_window,1))
    next_pred = model_FNN.predict(last_feature)
    dynamic_prediction = np.append(dynamic_prediction, next_pred)

# Transform forecast values to original scale
dynamic_prediction = dynamic_prediction.reshape(-1,1)
dynamic_prediction_original_scale = scaler.inverse_transform(dynamic_prediction)

# Plot
test_index = np.arange(len(data_scaled) - valid_size, len(data_scaled), 1)

plt.figure(figsize=(15,6))
plt.plot(scaler.inverse_transform(data_scaled[:len(data_scaled) - valid_size]), label='Training Data')
plt.plot(test_index, scaler.inverse_transform(data_scaled[-valid_size:]), label='Testing Data')
plt.plot(test_index, dynamic_prediction_original_scale[-valid_size:], label='Out of Sample Prediction') 
plt.legend(loc = "upper left")
plt.show()

In [ ]:
FNN_RMSE2 = math.sqrt(mean_squared_error(df2, dynamic_prediction_original_scale[-valid_size:]))
print('Test Data RMSE original scale: {0:.4f}'.format(FNN_RMSE2))

In [ ]:
rmse_data = {
    'Model': ['SARIMA', 'Holt-Winters', 'FNN'],
    'RMSE': [sarima_rmse22, hot_rmse2, FNN_RMSE2 ]}

gg2 = pd.DataFrame(rmse_data)
gg2_sorted = gg2.sort_values(by='RMSE')

print(gg2_sorted)

plt.figure(figsize=(14, 6))
bars = plt.barh(gg2_sorted['Model'], gg2_sorted['RMSE'], color='skyblue')

for bar in bars:
    width = bar.get_width()
    plt.text(width, bar.get_y() + bar.get_height() / 2, f'{width:.4f}', ha='left', va='center')
    
plt.xlabel('RMSE')
plt.title('RMSE Comparison')
plt.grid(axis='x')
plt.show()
# plt.savefig('RMSE_final')